In [180]:
#Use the requests library to get the overview page 
#and feed the HTML of that page to the BeautifulSoup library.


import requests
from bs4 import BeautifulSoup
req = requests.get("https://www.utrecht.nl/bestuur-en-organisatie/gemeenteraad/wie-zit-er-in-de-raad/")
html = req.text
soup = BeautifulSoup(html, "lxml")

In [182]:
#Use the select method of BeautifulSoup to make a list of the items containing the names, 
#whether that are table rows, list elements, divs, etcetera.

raadsleden_utrecht = [] 
   
overzicht = soup.select(".image")
site = "https://www.utrecht.nl" #url, needed to create links to detail personal pages


#Loop through that list and get the basic information out of the rows and append that as a dict to a list. 
#You should at least get the person's name and the link to their 'detail' page. 
#Optionally, limit this list to the first 25 people on the page.

for raadslid in overzicht[:25]:
    names = raadslid.find("a").get('title').split()
    firstname = names[0]
    lastname = names[1:]
    if "Fractievoorzitter" in lastname: #cleaning data, remove fractievoorzitter from lastname
        lastname.remove("Fractievoorzitter")
    lastname = " ".join(lastname)  #cleaning data, parring the lastnames
    
    url = raadslid.find("a").get('href')
    link = f"{site}{url}"

    
    raadsleden_utrecht.append({
        "firstname" : firstname,
        "lastname" : lastname,
        "link" : link
    })





In [183]:
#Convert the list with dicts to a Pandas dataframe, 
#save it to a CSV file and display the dataframe in the Notebook.

import pandas as pd
pd.DataFrame(raadsleden_utrecht)

df = pd.DataFrame(raadsleden_utrecht)

df.to_csv("gemeenteraadsleden_utrecht.csv")

In [184]:
#Loop through the list again, fetch the user detail page with requests, 
#parse that using BeautifulSoup and add user detail information to the list.

for raadslid in raadsleden_utrecht[:25]:
    req = requests.get(raadslid["link"])
    html = req.text
    soup = BeautifulSoup(html, "lxml")
       
    party = soup.select(".rs_skip")[1].select("a")[4].get_text()
    area = soup.select(".twocolumngroup-col2")[0].select("p")[0].get_text()

    subjects_span = soup.select(".layout-body")[2].select("p")
    if subjects_span:
        subjects = soup.select(".layout-body")[2].select("p")[0].get_text()
    else: 
        subjects = "none"
    
    raadslid["political party"] = party
    raadslid["lives in area"] = area
    raadslid["political topics"] = subjects



In [176]:
#Convert the ‘expanded’ list with dicts to a Pandas dataframa and save that to a CSV file, 
#and display the dataframe in the Notebook.

pd.DataFrame(raadsleden_utrecht)

df = pd.DataFrame(raadsleden_utrecht)

df.to_csv("gemeenteraadsleden_utrecht.csv")

df

,firstname,lastname,link,lives in area,political party,political topics
0,Heleen,de Boer,https://www.utrecht.nl/bestuur-en-organisatie/...,Lombok\r,GroenLinks,Fractievoorzitter GroenLinks\r
1,Peter,van Corler,https://www.utrecht.nl/bestuur-en-organisatie/...,Oog in Al\r,GroenLinks,"Jeugd en Jeugdzorg, Onderwijs, Regiozaken"
2,Fred,Dekkers,https://www.utrecht.nl/bestuur-en-organisatie/...,Muntkwartier\r,GroenLinks,"Welzijn, Wijkgericht werken/Participatie"
3,Melody,Deldjou Fard,https://www.utrecht.nl/bestuur-en-organisatie/...,Wijk C\r,GroenLinks,"Citymarketing, Cultuur, Economie, Diversiteit"
4,Janneke,van der Heijden,https://www.utrecht.nl/bestuur-en-organisatie/...,Vogelenbuurt\r,GroenLinks,"Internationale Zaken, Volksgezondheid, Zorg en MO"
5,Rachel,Heijne,https://www.utrecht.nl/bestuur-en-organisatie/...,Tuindorp Oost\r,GroenLinks,Plaatsvervangend voorzitter van de commissie M...
6,Julia,Kleinrensink,https://www.utrecht.nl/bestuur-en-organisatie/...,Tolsteeg\r,GroenLinks,Plaatsvervangend voorzitter van de commissie S...
7,Floor,de Koning,https://www.utrecht.nl/bestuur-en-organisatie/...,Abstede\r,GroenLinks,none
8,Sophie,Schers,https://www.utrecht.nl/bestuur-en-organisatie/...,Voordorp\r,GroenLinks,none
9,Erwin,Virginia,https://www.utrecht.nl/bestuur-en-organisatie/...,Plompetorengracht en omgeving\r,GroenLinks,none
